# Feature Selection On Gene Expression Data

<b>Data science problem </b>: Find a compact subset of features that is maximally predictive of the outcome of a classifier.<br>
<br>

Feature selection is particularly important for high-dimensionality and low-sample size datasets, and for datasets that contain redundant features, as a classificaton model built with relevant features will in general show higher class-discriminative power.<br>

In this analysis I will show how feature selection with Minimum Redundancy-Maximum Relevance (mRMR) can be used to improve classification performance of a Random Forest classifier on high-dimensional molecular data representing gene expression values. <br>

Random Forest (RF) is an enstablished machine-learning method for integrating gene expression data as it generally works well with high-dimensional problems. However, the presence of correlated features impacts RF’s ability to identify the strongest ones by decreasing the estimated importance scores of correlated features.  <br>
Training a RF classifier on a subset of relevant features selected by mRMR results in higher classification performance computed on a independent set. <br>


<br>

List of contents:<br>
1. Dataset Description.<br>
2. Load data.<br>
3. Prepare data.<br>
3.1 Standardize.<br>
3.1 Discretize.<br>
4. mRMR Feature Selection.<br>
5. Model Building and Classification.<br>
6. Resources

## 1. Dataset Description

I used the Golub *et al.* dataset from a proof-of-concept study published in 1999 [link to publication](https://science.sciencemag.org/content/286/5439/531/tab-article-info). The study showed how new cases of cancer could be classified by gene expression monitoring via DNA microarray. These data were used to classify patients with acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL).<br>
Gene expression data is an example of high-dimensionality and low-sample size dataset with high number of correlated feature vectors representing the gene expressions values. <br>
Feature selection applied to such dataset is used for biomarker discovery. <br>

## 2. Load data

In [ ]:
import pandas as pd


def read_data(path):
    """
    Helper function to read and preprocess original data.
    """
    df = pd.read_csv(path)
    cols = [c for c in df.columns if c.lower()[:4] != 'call']
    df = df[cols]

    del df['Gene Description']

    df = df.T
    df.columns = df.iloc[0] 
    return df[1:]

We use the helper function *read_data* to read original raw data, preprocess it, and transform it into pandas DataFrame. <br>
The *train* data will be used for feature selection and model building, while the *test* data will be used to compute the classification scores of the model.

In [73]:
# read train/test data
df_train = read_data('data/data_set_ALL_AML_train.csv')
df_test = read_data('data/data_set_ALL_AML_independent.csv')

# check that train and test sets have the same set of features
assert(set(df_train.columns) == set(df_test.columns))

# get feature names
features = df_train.columns

# read target data
y = pd.read_csv( 'data/actual.csv', index_col='patient')
y.index = [str(x) for x in y.index]

# split target vector between train and test
y_train = y.loc[df_train.index]
y_train = y_train['cancer'].to_list()

y_test = y.loc[df_test.index]
y_test = y_test['cancer'].to_list()

Let's printout some readout stats.

In [54]:
print('N. features =', df_train.shape[1])
classes = y['cancer'].unique()
print('target classes =', classes)

print()
print('Train set:')
print('N. samples =', df_train.shape[0])
print('N. %s =' %classes[0], y_train.count(classes[0]))
print('N. %s =' %classes[1], y_train.count(classes[1]))

print()
print('Test set:')
print('n. samples =', df_test.shape[0])
print('N. %s =' %classes[0], y_test.count(classes[0]))
print('N. %s =' %classes[1], y_test.count(classes[1]))

N. features = 7129
target classes = ['ALL' 'AML']

Train set:
N. samples = 38
N. ALL = 27
N. AML = 11

Test set:
n. samples = 34
N. ALL = 20
N. AML = 14


We note that both train and test sets are unbalanced, but we will see later how the RF classifier will take care of it. <br>
Let’s display the first 5 rows of the train DataFrame:

In [29]:
df_train.head()

Gene Accession Number,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,AFFX-BioB-5_st,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
1,-214,-153,-58,88,-295,-558,199,-176,252,206,...,185,511,-125,389,-37,793,329,36,191,-37
2,-139,-73,-1,283,-264,-400,-330,-168,101,74,...,169,837,-36,442,-17,782,295,11,76,-14
3,-76,-49,-307,309,-376,-650,33,-367,206,-215,...,315,1199,33,168,52,1138,777,41,228,-41
4,-135,-114,265,12,-419,-585,158,-253,49,31,...,240,835,218,174,-110,627,170,-50,126,-91
5,-106,-125,-76,168,-230,-284,4,-122,70,252,...,156,649,57,504,-26,250,314,14,56,-25


The data shows gene expression values for 7129 different genes. <br>
We will now extract *train* and *test* feature matrices out of the relative DataFrames.

In [34]:
X_train = df_train.values
X_test = df_test.values

## 3. Prepare Data

### 3.1 Standardize


We use scikit-learn StandardScaler class to standardize our train and test data.

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

### 3.2 Discretize


Since the mRMR algorithm makes use of information metric functions, such as Shanon Entropy, we require the data to be discretized.<br>
The discretization procedure aims at partitioning continuous variables to discretized intervals so that we can compute information metrics on. <br>

Here we chose to use the scikit-learn KBinsDiscretizer for constant-width bins discretization that 
splits, for each gene, the full range of ob-served gene expression values in bins with equal size.

In [39]:
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
X_train_discr = discretizer.fit_transform(X_train)

## 4. mRMR Feature Selection

Once we have our training set discretized, we can fit our mRMR algorithm on. We decide to select *5* features.

In [86]:
from mrmr import MRMR

mrmr = MRMR(n_features=5)
selected_indices = mrmr.fit(X_train_discr, y_train)
selected_feature_names = features[selected_indices]

print('selected feature indices', selected_feature_names.to_list())

selected feature indices ['X95735_at', 'Y12670_at', 'M55150_at', 'U50136_rna1_at', 'M21551_rna1_at']


We now filter the *train* and *test* feature matrices according to the selected feature indices

In [ ]:
X_train_selection = X_train[:, selected_indices]
X_test_selection = X_test[:, selected_indices]

## 5. Model Building and Classification

We will now build three different RF classifiers:
1. the first one will be trained with the origianl feature matrix $X$,
2. the second one will be trained with the feature matrix $X_{selection}$ containing only selected feature fectors,
3. the third one will be trained with the feature matrix $X_{random}$ containing 5 random feature vectors. <br>

Note: as we saw above, in both *train* and *test* set the data is unbalanced towards the subclass ALL. By setting the *class_weight* param to “balanced” mode we tell RandomForestClassifier to to automatically adjust weights inversely proportional to class frequencies in the input data.

In [ ]:
import numpy as np
np.random.seed(123)

In [91]:
from sklearn.ensemble import RandomForestClassifier

print("Accuracy on test data:")

# classify with all features
clf1 = RandomForestClassifier(class_weight='balanced')
clf1.fit(X_train, y_train)
print("All features: {:.2f}".format(clf1.score(X_test, y_test)))

# classify with selected features
clf2 = RandomForestClassifier(class_weight='balanced')
clf2.fit(X_train_selection, y_train)
print("mRMR features: {:.2f}".format(clf2.score(X_test_selection, y_test)))

# classify with random features
clf3 = RandomForestClassifier(class_weight='balanced')
clf3.fit(X_train[:, -5:], y_train)
print("Random features: {:.2f}".format(clf3.score(X_test[:, -5:], y_test)))

Accuracy on test data:
All features: 0.74
mRMR features: 0.85
Random features: 0.53


As we can see, the accuracy score of RF classifier build with the $X_{selection}$ outperforms the other two models.

## 6. Resources

You can download the Notebook from my [GitHub page](https://github.com/aleromualdi/mRMR)